In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
import math
import torch
import re

In [2]:
data = pandas.read_csv('./data/Vout_new_negwt.csv')
data_split = data.iloc[:, 1:]
data_split

,"/VOUT (R1_data=1600000,R2_data=12500,W1_data=8.5e-05,L1_data=7e-05,W2_data=0.000495,L2_data=3.2e-05,R_inv=350000,W_inv=0.00017,L_inv=0.000125) Y","/VOUT (R1_data=700000,R2_data=25500,W1_data=9.5e-05,L1_data=5e-05,W2_data=0.000485,L2_data=3.8e-05,R_inv=450000,W_inv=0.00011,L_inv=7.5e-05) Y","/VOUT (R1_data=925000,R2_data=15750,W1_data=9.8e-05,L1_data=5.5e-05,W2_data=0.000482,L2_data=3.4e-05,R_inv=425000,W_inv=0.000185,L_inv=0.000138) Y","/VOUT (R1_data=1825000,R2_data=28750,W1_data=8.8e-05,L1_data=7.5e-05,W2_data=0.000492,L2_data=3.9e-05,R_inv=325000,W_inv=0.000125,L_inv=8.8e-05) Y","/VOUT (R1_data=1375000,R2_data=9250,W1_data=9.2e-05,L1_data=6.5e-05,W2_data=0.000498,L2_data=3.1e-05,R_inv=475000,W_inv=9.5e-05,L_inv=6.2e-05) Y","/VOUT (R1_data=475000,R2_data=22250,W1_data=8.2e-05,L1_data=4.5e-05,W2_data=0.000488,L2_data=3.6e-05,R_inv=375000,W_inv=0.000155,L_inv=0.000112) Y","/VOUT (R1_data=587500,R2_data=14125,W1_data=8.9e-05,L1_data=6.2e-05,W2_data=0.000486,L2_data=3.4e-05,R_inv=487500,W_inv=0.000192,L_inv=0.000144) Y","/VOUT (R1_data=1487500,R2_data=27125,W1_data=9.9e-05,L1_data=4.2e-05,W2_data=0.000496,L2_data=3.9e-05,R_inv=387500,W_inv=0.000132,L_inv=9.4e-05) Y","/VOUT (R1_data=1937500,R2_data=7625,W1_data=8.4e-05,L1_data=5.2e-05,W2_data=0.000491,L2_data=3.2e-05,R_inv=437500,W_inv=0.000102,L_inv=6.9e-05) Y","/VOUT (R1_data=1037500,R2_data=20625,W1_data=9.4e-05,L1_data=7.2e-05,W2_data=0.000481,L2_data=3.7e-05,R_inv=337500,W_inv=0.000162,L_inv=0.000119) Y",...,"/VOUT (R1_data=483240,R2_data=25083,W1_data=8.3e-05,L1_data=6.6e-05,W2_data=0.000489,L2_data=3.5e-05,R_inv=309045,W_inv=0.000115,L_inv=0.000117) Y","/VOUT (R1_data=595740,R2_data=13708,W1_data=8.9e-05,L1_data=4.4e-05,W2_data=0.000485,L2_data=3.7e-05,R_inv=421545,W_inv=9.2e-05,L_inv=0.000148) Y","/VOUT (R1_data=1495740,R2_data=26708,W1_data=9.9e-05,L1_data=6.4e-05,W2_data=0.000495,L2_data=3.2e-05,R_inv=321545,W_inv=0.000152,L_inv=9.8e-05) Y","/VOUT (R1_data=1945740,R2_data=7208,W1_data=8.4e-05,L1_data=7.4e-05,W2_data=0.00049,L2_data=3.9e-05,R_inv=471545,W_inv=0.000182,L_inv=7.3e-05) Y","/VOUT (R1_data=1045740,R2_data=20208,W1_data=9.4e-05,L1_data=5.4e-05,W2_data=0.00048,L2_data=3.4e-05,R_inv=371545,W_inv=0.000122,L_inv=0.000123) Y","/VOUT (R1_data=820740,R2_data=10458,W1_data=9.2e-05,L1_data=5.9e-05,W2_data=0.000488,L2_data=3.8e-05,R_inv=346545,W_inv=0.000197,L_inv=6.1e-05) Y","/VOUT (R1_data=1720740,R2_data=23458,W1_data=8.2e-05,L1_data=7.9e-05,W2_data=0.000498,L2_data=3.3e-05,R_inv=446545,W_inv=0.000137,L_inv=0.000111) Y","/VOUT (R1_data=1270740,R2_data=16958,W1_data=9.7e-05,L1_data=6.9e-05,W2_data=0.000493,L2_data=3.6e-05,R_inv=396545,W_inv=0.000107,L_inv=0.000136) Y","/VOUT (R1_data=370740,R2_data=29958,W1_data=8.7e-05,L1_data=4.9e-05,W2_data=0.000483,L2_data=3.1e-05,R_inv=496545,W_inv=0.000167,L_inv=8.6e-05) Y","/VOUT (R1_data=250000,R2_data=6000,W1_data=8e-05,L1_data=4e-05,W2_data=0.00048,L2_data=3e-05,R_inv=300000,W_inv=8e-05,L_inv=5e-05) Y"
0,0.7068,0.9972,0.8618,0.9985,-0.06775,0.9961,0.7721,0.9977,-0.8753,0.9777,...,0.9994,0.8478,0.9995,-0.9468,0.9915,-0.8650,0.9979,0.929,0.9996,-0.7699
1,0.7066,0.9972,0.8618,0.9985,-0.06775,0.9961,0.7721,0.9977,-0.8753,0.9777,...,0.9994,0.8478,0.9995,-0.9468,0.9915,-0.8647,0.9979,0.929,0.9996,-0.7698
2,0.7066,0.9972,0.8618,0.9985,-0.06775,0.9961,0.7721,0.9977,-0.8753,0.9777,...,0.9994,0.8478,0.9995,-0.9468,0.9915,-0.8647,0.9979,0.929,0.9996,-0.7698
3,0.7066,0.9972,0.8618,0.9985,-0.06775,0.9961,0.7721,0.9977,-0.8753,0.9777,...,0.9994,0.8478,0.9995,-0.9468,0.9915,-0.8647,0.9979,0.929,0.9996,-0.7698
4,0.7066,0.9972,0.8618,0.9985,-0.06775,0.9961,0.7721,0.9977,-0.8753,0.9777,...,0.9994,0.8478,0.9995,-0.9468,0.9915,-0.8647,0.9979,0.929,0.9996,-0.7698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,-0.9060,-0.9313,-0.9209,-0.9033,-0.93750,-0.9135,-0.9303,-0.9173,-0.9300,-0.9028,...,-0.8565,-0.8355,-0.9101,-0.9609,-0.8803,-0.9589,-0.9183,-0.860,-0.9525,-0.9068
397,-0.9060,-0.9313,-0.9209,-

In [3]:
datalist=data_split.columns.values.tolist()
print(datalist)
print(len(datalist))

['/VOUT (R1_data=1600000,R2_data=12500,W1_data=8.5e-05,L1_data=7e-05,W2_data=0.000495,L2_data=3.2e-05,R_inv=350000,W_inv=0.00017,L_inv=0.000125) Y', '/VOUT (R1_data=700000,R2_data=25500,W1_data=9.5e-05,L1_data=5e-05,W2_data=0.000485,L2_data=3.8e-05,R_inv=450000,W_inv=0.00011,L_inv=7.5e-05) Y', '/VOUT (R1_data=925000,R2_data=15750,W1_data=9.8e-05,L1_data=5.5e-05,W2_data=0.000482,L2_data=3.4e-05,R_inv=425000,W_inv=0.000185,L_inv=0.000138) Y', '/VOUT (R1_data=1825000,R2_data=28750,W1_data=8.8e-05,L1_data=7.5e-05,W2_data=0.000492,L2_data=3.9e-05,R_inv=325000,W_inv=0.000125,L_inv=8.8e-05) Y', '/VOUT (R1_data=1375000,R2_data=9250,W1_data=9.2e-05,L1_data=6.5e-05,W2_data=0.000498,L2_data=3.1e-05,R_inv=475000,W_inv=9.5e-05,L_inv=6.2e-05) Y', '/VOUT (R1_data=475000,R2_data=22250,W1_data=8.2e-05,L1_data=4.5e-05,W2_data=0.000488,L2_data=3.6e-05,R_inv=375000,W_inv=0.000155,L_inv=0.000112) Y', '/VOUT (R1_data=587500,R2_data=14125,W1_data=8.9e-05,L1_data=6.2e-05,W2_data=0.000486,L2_data=3.4e-05,R_inv

In [4]:
def extract_values(data_list):
    patterns = [
        r"R1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"R2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"R_inv=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"W1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L1_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"W2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L2_data=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"W_inv=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))",
        r"L_inv=(-?\d+.?\d*(?:e-?\d+)?)(?=,|\))"
    ]
    
    matrix = []
    
    for data in data_list:
        row = []
        for pattern in patterns:
            match = re.search(pattern, data)
            if match:
                value = match.group(1)
                if 'e' in value or '.' in value: 
                    row.append(float(value))
                else:
                    row.append(int(value))
            else:
                row.append(None)
        matrix.append(row)
    return matrix

result = extract_values(datalist)

In [5]:
result_temp = torch.tensor(result)
result_temp.max(0)[0]

tensor([2.0498e+06, 3.1997e+04, 4.9998e+05, 1.0000e-04, 8.0000e-05, 5.0000e-04,
        4.0000e-05, 2.0000e-04, 1.5000e-04])

In [6]:
result_temp.min(0)[0]

tensor([2.5000e+05, 6.0000e+03, 3.0000e+05, 8.0000e-05, 4.0000e-05, 4.8000e-04,
        3.0000e-05, 8.0000e-05, 5.0000e-05])

In [7]:
torch.save(result_temp, './data/neg_RT.data')

In [11]:
r1 = result_temp[:, 0].view(-1,1)
r2 = result_temp[:, 1].view(-1,1)
r3 = result_temp[:, 2].view(-1,1)
w1 = result_temp[:, 3].view(-1,1)
l1 = result_temp[:, 4].view(-1,1)
w2 = result_temp[:, 5].view(-1,1)
l2 = result_temp[:, 6].view(-1,1)
w3 = result_temp[:, 7].view(-1,1)
l3 = result_temp[:, 8].view(-1,1)

ratio_1 = w1/l1
ratio_2 = w2/l2
ratio_3 = w3/l3

r1.shape, r2.shape, r3.shape, w1.shape, l1.shape, w2.shape, l2.shape, w3.shape, l3.shape, ratio_1.shape, ratio_2.shape, ratio_3.shape

(torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]),
 torch.Size([9999, 1]))

In [12]:
C = torch.cat([r1, r2, r3, w1, l1, w2, l2, w3, l3, ratio_1, ratio_2, ratio_3], dim=1)
C.shape

torch.Size([9999, 12])

In [13]:
torch.save(C, './data/neg_RT_ratio.data')